In [1]:
import sys

sys.path.insert(0, sys.path[0].removesuffix("/src/jupyter_nb"))

import pandas as pd
import src.utils as utils
import src.calcEMA as calc_utils
import src.myenv as myenv
from datetime import datetime
import numpy as np

<built-in method __dir__ of sqlite3.Connection object at 0x7fb9db396740>


In [2]:
data = utils.get_data(
    symbol="BTCUSDT",
    save_database=False,
    interval="1h",
    tail=-1,
    columns=myenv.all_klines_cols,
    parse_dates=True,
)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 55405 entries, 2017-08-17 04:00:00 to 2023-12-18 00:00:00
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     55405 non-null  datetime64[ns]
 1   open                          55405 non-null  float32       
 2   high                          55405 non-null  float32       
 3   low                           55405 non-null  float32       
 4   close                         55405 non-null  float32       
 5   volume                        55405 non-null  float32       
 6   close_time                    55405 non-null  int64         
 7   quote_asset_volume            55405 non-null  float32       
 8   number_of_trades              55405 non-null  int16         
 9   taker_buy_base_asset_volume   55405 non-null  float32       
 10  taker_buy_quote_asset_volume  55405 non-null  float32      

In [3]:
_range_p_ema_ini = 50
_range_p_ema_end = 250
_step_p_ema = 25
data = calc_utils.calc_RSI(data)
data = calc_utils.calc_ema_periods(data, periods_of_time=[i for i in range(_range_p_ema_ini, _range_p_ema_end + 1, _step_p_ema)], diff_price=False)
data = utils.truncate_data_in_days(data, myenv.days_to_validate_train)
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2161 entries, 2023-09-19 00:00:00 to 2023-12-18 00:00:00
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   open_time                     2161 non-null   datetime64[ns]
 1   open                          2161 non-null   float32       
 2   high                          2161 non-null   float32       
 3   low                           2161 non-null   float32       
 4   close                         2161 non-null   float32       
 5   volume                        2161 non-null   float32       
 6   close_time                    2161 non-null   int64         
 7   quote_asset_volume            2161 non-null   float32       
 8   number_of_trades              2161 non-null   int16         
 9   taker_buy_base_asset_volume   2161 non-null   float32       
 10  taker_buy_quote_asset_volume  2161 non-null   float32       

In [4]:
_symbol_list = ['BTCUSDT']
_interval_list = ['1m']
_target_margin_list = [1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0 ]
_min_rsi = 28
_max_rsi = 80

_data = data

count = 0
r = {}

for interval in _interval_list:
    for symbol in _symbol_list:
        ix_symbol = f'{symbol}_{interval}'
        for p_ema in range(_range_p_ema_ini, _range_p_ema_end + 1, 25):
            for min_rsi in range(_min_rsi, 36 + 1, 2):
                for max_rsi in range(72, _max_rsi + 1, 2):
                    ix_predict = f'{symbol}_{interval}_{p_ema}_{min_rsi}_{max_rsi}'
                    _data[ix_predict] = np.where( (_data['close'] > _data[f'ema_{p_ema}p']) & (_data['rsi'] >= max_rsi) , 'SHORT', np.where((_data['close'] < _data[f'ema_{p_ema}p']) & (_data['rsi'] <= min_rsi), 'LONG', 'ESTAVEL')) 
                    for target_margin in _target_margin_list:
                        for stop_loss_multiplier in range(2, myenv.stop_loss_range_multiplier + 1):
                            count += 1
                            ix = f'{symbol}_{interval}_{target_margin}_{p_ema}_{min_rsi}_{max_rsi}_{stop_loss_multiplier}'
                            r[f'{ix}_purchased'] = False
                            r[f'{ix}_perform_sell'] = False
                            r[f'{ix}_purchase_price'] = 0.0
                            r[f'{ix}_purchase_strategy'] = ''
                            r[f'{ix}_start_amount_invested'] = 100.0
print(f'count: {count}')

count: 12150


In [5]:
margin_operation = 0.0
count = 0
for _, row in _data.iterrows():
	count += 1
	if count % 100 == 0:
		print(f'count: {count}')
	actual_price = row['close']
	rsi = row['rsi']
	p_ema_value = row[f'ema_{p_ema}p']
	'''
	open_time = row['open_time']	
	if isinstance(open_time, np.datetime64):
		_open_time = pd.to_datetime(open_time, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
	else:
		_open_time = open_time.strftime('%Y-%m-%d %H:%M:%S')
	'''
	for interval in _interval_list:
		for symbol in _symbol_list:
			for p_ema in range(_range_p_ema_ini, _range_p_ema_end + 1, 25):
				for min_rsi in range(_min_rsi, 36 + 1, 2):
					for max_rsi in range(72, _max_rsi + 1, 2):
						ix_predict = f'{symbol}_{interval}_{p_ema}_{min_rsi}_{max_rsi}'
						strategy = row[ix_predict]
						for target_margin in _target_margin_list:
							for stop_loss_multiplier in range(2, myenv.stop_loss_range_multiplier + 1):
								ix = f'{symbol}_{interval}_{target_margin}_{p_ema}_{min_rsi}_{max_rsi}_{stop_loss_multiplier}'
								if not r[f'{ix}_purchased']:
									if strategy.startswith('LONG') or strategy.startswith('SHORT'):  # If true, BUY
										r[f'{ix}_purchased'] = True
										r[f'{ix}_purchase_price'] = actual_price
										r[f'{ix}_purchase_strategy'] = strategy
										r[f'{ix}_take_profit'], r[f'{ix}_stop_loss'] = utils.calc_take_profit_stop_loss(strategy, r[f'{ix}_purchase_price'], target_margin, stop_loss_multiplier)
#										print(f"BUY[{_open_time,}]: {r[f'{ix}_purchase_strategy']} - Price: {actual_price:.6f} - Target Margin: {target_margin:.2f}% - Take Profit: \
#{r[f'{ix}_take_profit']:.2f} - Stop Loss: {r[f'{ix}_stop_loss']:.2f} - RSI: {rsi:.2f}% - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi}% - Max RSI: {max_rsi}% - Stop Loss Multiplier: {stop_loss_multiplier}")
										continue

								if r[f'{ix}_purchased']:  # and (operation.startswith('LONG') or operation.startswith('SHORT')):
									# print(f'[{row_nu}][{strategy}] => Purchased: {purchased} - Price: {actual_price:.6f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Min RSI: {min_rsi} - Max RSI: {max_rsi}')
									if r[f'{ix}_purchase_strategy'].startswith('LONG'):
										margin_operation = (actual_price - r[f'{ix}_purchase_price']) / r[f'{ix}_purchase_price']
										if ((actual_price >= r[f'{ix}_take_profit']) or (actual_price <= r[f'{ix}_stop_loss'])):  # Long ==> Sell - Take Profit / Stop Loss
											r[f'{ix}_perform_sell']  = True
									elif r[f'{ix}_purchase_strategy'].startswith('SHORT'):
										margin_operation = (r[f'{ix}_purchase_price'] - actual_price) / r[f'{ix}_purchase_price']
										if ((actual_price <= r[f'{ix}_take_profit']) or (actual_price >= r[f'{ix}_stop_loss'])):  # Short ==> Sell - Take Profit / Stop Loss
											r[f'{ix}_perform_sell']  = True

									if r[f'{ix}_perform_sell']:
										r[f'{ix}_start_amount_invested'] = (1 + margin_operation) * r[f'{ix}_start_amount_invested']
#										print(f"SELL[{_open_time}]: {r[f'{ix}_purchase_strategy']} - Price: {actual_price:.6f} - Purchase Price: {r[f'{ix}_purchase_price']:.6f} - Target Margin: {target_margin:.2f}% - Margin Operation:\
#{margin_operation*100:.2f}% - Take Profit: {r[f'{ix}_take_profit']:.2f} - Stop Loss: {r[f'{ix}_stop_loss']:.2f} - RSI: {rsi:.2f} - ema_{p_ema}p: {p_ema_value:.2f} - Sum PnL: {r[f'{ix}_start_amount_invested']:.2f}  - Stop Loss Multiplier: {stop_loss_multiplier}")
										r[f'{ix}_perform_sell']  = False
										r[f'{ix}_purchase_strategy'] = ''
										r[f'{ix}_purchased'] = False

count: 100
count: 200
count: 300
count: 400
count: 500
count: 600
count: 700
count: 800
count: 900
count: 1000
count: 1100
count: 1200
count: 1300
count: 1400
count: 1500
count: 1600
count: 1700
count: 1800
count: 1900
count: 2000
count: 2100


In [7]:
margin_operation = 0.0
for interval in _interval_list:
	for symbol in _symbol_list:
		for target_margin in _target_margin_list:
			for p_ema in range(_range_p_ema_ini, _range_p_ema_end + 1, 25):
				for min_rsi in range(_min_rsi, 36 + 1, 2):
					for max_rsi in range(72, _max_rsi + 1, 2):
						ix_predict = f'{p_ema}_{min_rsi}_{max_rsi}'
						for stop_loss_multiplier in range(2, myenv.stop_loss_range_multiplier + 1):
							ix = f'{symbol}_{interval}_{target_margin}_{p_ema}_{min_rsi}_{max_rsi}_{stop_loss_multiplier}'
							print(f"{ix} - Result Simule Trading: {r[f'{ix}_start_amount_invested']:.2f}")
								

BTCUSDT_1m_1.0_50_28_72_2 - Result Simule Trading: 86.79
BTCUSDT_1m_1.0_50_28_72_3 - Result Simule Trading: 83.30
BTCUSDT_1m_1.0_50_28_72_4 - Result Simule Trading: 85.60
BTCUSDT_1m_1.0_50_28_72_5 - Result Simule Trading: 85.82
BTCUSDT_1m_1.0_50_28_72_6 - Result Simule Trading: 95.14
BTCUSDT_1m_1.0_50_28_72_7 - Result Simule Trading: 91.88
BTCUSDT_1m_1.0_50_28_74_2 - Result Simule Trading: 85.82
BTCUSDT_1m_1.0_50_28_74_3 - Result Simule Trading: 84.39
BTCUSDT_1m_1.0_50_28_74_4 - Result Simule Trading: 87.43
BTCUSDT_1m_1.0_50_28_74_5 - Result Simule Trading: 84.98
BTCUSDT_1m_1.0_50_28_74_6 - Result Simule Trading: 92.93
BTCUSDT_1m_1.0_50_28_74_7 - Result Simule Trading: 90.20
BTCUSDT_1m_1.0_50_28_76_2 - Result Simule Trading: 84.14
BTCUSDT_1m_1.0_50_28_76_3 - Result Simule Trading: 80.70
BTCUSDT_1m_1.0_50_28_76_4 - Result Simule Trading: 94.98
BTCUSDT_1m_1.0_50_28_76_5 - Result Simule Trading: 86.87
BTCUSDT_1m_1.0_50_28_76_6 - Result Simule Trading: 88.51
BTCUSDT_1m_1.0_50_28_76_7 - Res